In [1]:
import os
import re
import json
import pandas as pd
from textwrap import wrap
from openai import OpenAI
from prettytable import PrettyTable

### Get annotated data:

In [2]:
df = pd.read_csv("../220614_GreatSchools_Revised_Annotated_Data.csv", low_memory=False)
df.head(3)

,Unnamed: 0,sch_ID,sch_name,sch_state,sch_county,status,sch_type,sch_level,sch_mean_ach,sch_grade_slope_mean_ach,...,L2_facilities,L2_religious,L2_school choice/enrollment,L2_building quality,L2_students,L2_curriculum/curricular materials,L2_school staff,L2_school finances,L2_school type,L2_student discipline
0,0,10000500870,Albertville Middle School,AL,MARSHALL COUNTY,1.0,tps,Middle,-0.331363,NaN,...,1,0,1,0,1,1,1,0,0,0
1,1,10000500871,ALBERTVILLE HIGH SCH,AL,MARSHALL COUNTY,1.0,tps,High,NaN,NaN,...,1,0,1,0,1,1,0,0,1,0
2,2,10000500871,ALBERTVILLE HIGH SCH,AL,MARSHALL COUNTY,1.0,tps,High,NaN,NaN,...,0,0,0,0,1,0,1,0,0,0


In [3]:
df.columns

Index(['Unnamed: 0', 'sch_ID', 'sch_name', 'sch_state', 'sch_county', 'status',
       'sch_type', 'sch_level', 'sch_mean_ach', 'sch_grade_slope_mean_ach',
       'sch_star_rating_avg', 'resp_count', 'sy_text', 'who', 'comments',
       'review_star_rating', 'sch_gs_rating', 'words', 'is_charter',
       'is_private', 'is_tps', 'is_good_rev', 'is_bad_rev', 'is_good_sch',
       'is_bad_sch', 'no_rev_rating', 'no_sch_rating', 'L1_school staff',
       'L1_school culture', 'L1_resources', 'L1_instruction & learning',
       'L1_school-level features', 'L1_physical environment',
       'L1_overall quality', 'L1_other', 'L2_postsecondary/graduation',
       'L2_school culture', 'L2_school safety', 'L2_preparation',
       'L2_location', 'L2_learning experience', 'L2_quality indicator',
       'L2_instruction', 'L2_evaluation', 'L2_family', 'L2_other',
       'L2_extracurricular/electives', 'L2_teacher quality', 'L2_offering',
       'L2_interpersonal relationships', 'L2_facilities', 'L2_re

### Get categories from annotated data (to compare later)

In [4]:
# l1_vars = [column for column in df.columns if "L1" in column]
l2_vars = [column for column in df.columns if "L2" in column]
l2_vars


['L2_postsecondary/graduation',
 'L2_school culture',
 'L2_school safety',
 'L2_preparation',
 'L2_location',
 'L2_learning experience',
 'L2_quality indicator',
 'L2_instruction',
 'L2_evaluation',
 'L2_family',
 'L2_other',
 'L2_extracurricular/electives',
 'L2_teacher quality',
 'L2_offering',
 'L2_interpersonal relationships',
 'L2_facilities',
 'L2_religious',
 'L2_school choice/enrollment',
 'L2_building quality',
 'L2_students',
 'L2_curriculum/curricular materials',
 'L2_school staff',
 'L2_school finances',
 'L2_school type',
 'L2_student discipline']

### Get categores from keywords file (source of truth?)

In [5]:
categories = list(set([category.strip() for category in list(pd.read_csv("../Final_Keywords_LL_Mar152022.csv")["Level 2 Focused Category (from Topic-Based Framework & some new)"].unique())]))
categories

['Student discipline',
 'Interpersonal relationships',
 'Students',
 'Teacher quality',
 'School culture',
 'Facilities',
 'Preparation',
 'Religious',
 'School type',
 'Location',
 'Evaluation',
 'Curriculum/curricular materials',
 'Offering',
 'School finances',
 'Postsecondary/graduation',
 'School choice/enrollment',
 'Learning Experience',
 'Politics',
 'Building quality',
 'Extracurricular/electives',
 'Quality indicator',
 'Other',
 'Family',
 'School staff',
 'Instruction',
 'School safety']

### Map from keywords file to the column names in annotation file:

In [6]:
# from ChatGPT: 
mapping = {
    'Postsecondary/graduation': 'L2_postsecondary/graduation',
    'School culture': 'L2_school culture',
    'School safety': 'L2_school safety',
    'Preparation': 'L2_preparation',
    'Location': 'L2_location',
    'Learning Experience': 'L2_learning experience',
    'Quality indicator': 'L2_quality indicator',
    'Instruction': 'L2_instruction',  
    'Evaluation': 'L2_evaluation',
    'Family': 'L2_family',
    'Other': 'L2_other',
    'Extracurricular/electives': 'L2_extracurricular/electives',
    'Teacher quality': 'L2_teacher quality',
    'Offering': 'L2_offering',
    'Interpersonal relationships': 'L2_interpersonal relationships',
    'Facilities': 'L2_facilities',
    'Religious': 'L2_religious',
    'School choice/enrollment': 'L2_school choice/enrollment',
    'Building quality': 'L2_building quality', 
    'Students': 'L2_students',
    'Curriculum/curricular materials': 'L2_curriculum/curricular materials',
    'School staff': 'L2_school staff',
    'School finances': 'L2_school finances',
    'School type': 'L2_school type',
    'Student discipline': 'L2_student discipline'
}

## Test:

Get a single comment from annotations and query gpt for annotations, then compare against the annotation from annotations file.

In [7]:
comment = df.iloc[0].comments
comment

"(PLEASE DONOT SEND YOUR KIDS HERE) LOTS OF BULLYING AND FIGHTS I went here when I was a kid and it was bad and now it is worst as it has ever been I well adment the teachers are ok I loved Mrs.Long and Mrs.Right but there were lots of fights and lots of people in my class was below grade level and had 50's,40's,30's,and even F's SO I would just like everyone to know about this school.Thanks"

In [21]:
client = OpenAI() # defaults to os.environ["OPENAI_API_KEY"]

def query_gpt(categories, comment, model="gpt-3.5-turbo"):

    query = "Which of these topics: " + str(categories) + f'\n\nAre discussed in this review: "{comment}". Respond in the format: "topic": 0 or 1'

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role":"user", "content":query}
        ]
    )
    response = completion.choices[0].message.content
    return response
    
response35 = query_gpt(categories, comment)
print("3.5:", response35)
response40 = query_gpt(categories, comment, "gpt-4")
print("4.0:",response40)

3.5: 'Student discipline': 1
'Interpersonal relationships': 0
'Students': 1
'Teacher quality': 1
'School culture': 0
'Facilities': 0
'Preparation': 0
'Religious': 0
'School type': 0
'Location': 0
'Evaluation': 0
'Curriculum/curricular materials': 0
'Offering': 0
'School finances': 0
'Postsecondary/graduation': 0
'School choice/enrollment': 0
'Learning Experience': 0
'Politics': 0
'Building quality': 0
'Extracurricular/electives': 0
'Quality indicator': 0
'Other': 0
'Family': 0
'School staff': 0
'Instruction': 0
'School safety': 0
4.0: {"Student discipline": 1, "Interpersonal relationships": 0, "Students": 1, "Teacher quality": 1, "School culture": 1, "Facilities": 0, "Preparation": 0, "Religious": 0, "School type": 0, "Location": 0, "Evaluation": 0, "Curriculum/curricular materials": 1, "Offering": 0, "School finances": 0, "Postsecondary/graduation": 0, "School choice/enrollment": 0, "Learning Experience": 1, "Politics": 0, "Building quality": 0, "Extracurricular/electives": 0, "Qualit

In [33]:
dict40 = json.loads(response40)

predicted_values35 = [ [val.strip("'") for val in topic.split(": ")] for topic in response35.split('\n') ]
predicted_values40 = [[val, dict40[val]] for val in list(dict40.keys())]

### Compare using jaccard similarity

In [35]:
def jaccard_similarity(vector1, vector2):
    if len(vector1) != len(vector2):
        raise ValueError("Vectors must be of equal length")

    intersection = sum(int(el1) and int(el2) for el1, el2 in zip(vector1, vector2))
    union = sum(int(el1) or int(el2) for el1, el2 in zip(vector1, vector2))

    if union == 0:
        return 0.0  # to handle the case where both vectors are empty
    else:
        return intersection / union

In [37]:
print("\t ChatGPT | Annotation")

categories_list, annotations, gpt_predictions35, gpt_predictions40 = [], [], [], []

for prediction35, prediction40 in zip(predicted_values35, predicted_values40):
    if prediction35[0] in mapping and prediction40[0] in mapping:
        df_name = mapping[prediction35[0]]
    else:
        print(f"{prediction35[0]} or {prediction40} not in DF. Skipping.")
        continue

    categories_list.append(prediction35[0])
    annotations.append(df.iloc[0][df_name])
    gpt_predictions35.append(prediction35[1])
    gpt_predictions40.append(prediction40[1])
    
table = PrettyTable()
table.add_column("category", categories_list)
table.add_column("hum", annotations)
table.add_column("3.5", gpt_predictions35)
table.add_column("4.0", gpt_predictions40)

print(table)
print(f"Jaccard similarity 3.5: {jaccard_similarity(annotations, gpt_predictions35):}")
print(f"Jaccard similarity 4.0: {jaccard_similarity(annotations, gpt_predictions40):}")

	 ChatGPT | Annotation
Politics or ['Politics', 0] not in DF. Skipping.
+---------------------------------+-----+-----+-----+
|             category            | hum | 3.5 | 4.0 |
+---------------------------------+-----+-----+-----+
|        Student discipline       |  0  |  1  |  1  |
|   Interpersonal relationships   |  1  |  0  |  0  |
|             Students            |  1  |  1  |  1  |
|         Teacher quality         |  0  |  1  |  1  |
|          School culture         |  0  |  0  |  1  |
|            Facilities           |  1  |  0  |  0  |
|           Preparation           |  0  |  0  |  0  |
|            Religious            |  0  |  0  |  0  |
|           School type           |  0  |  0  |  0  |
|             Location            |  0  |  0  |  0  |
|            Evaluation           |  1  |  0  |  0  |
| Curriculum/curricular materials |  1  |  0  |  1  |
|             Offering            |  0  |  0  |  0  |
|         School finances         |  0  |  0  |  0  |
|     Post

In [ ]:
print("\n".join(wrap(text=comment, width=100)))

(PLEASE DONOT SEND YOUR KIDS HERE) LOTS OF BULLYING AND FIGHTS I went here when I was a kid and it
was bad and now it is worst as it has ever been I well adment the teachers are ok I loved Mrs.Long
and Mrs.Right but there were lots of fights and lots of people in my class was below grade level and
had 50's,40's,30's,and even F's SO I would just like everyone to know about this school.Thanks


## Takeaways:

GPT is not going to give us the same values human annotators write in, but that doesn't mean the results should be discarded.


On an infrastructure level, this problem is easily solvable by ChatGPT. The problem of applying this extremely versatile model to a dataset is one of prompt-engineering. Each token incurs a computational and financial cost- prompts need to be concise but specific in how the model should format it's answer, especially considering the application of this model to thousands of records. 


### Quality of GPT categorization:

gpt 3.5 has satisfactory (but also somewhat different) performance from human annotators. gpt 4.0 performs better than human annotation in the test review I fed it, and makes a compelling case for the elements included/not included.

Either way, GPT predictions don't match human labels to the letter. But I'd argue they're still valuable. 4.0 is probably the way to go.


#### ChatGPT's case:

In [ ]:
def query_gpt(categories, comment):

    query = "Which of these topics: " + str(categories) + f'\n\nAre discussed in this review: "{comment}". Respond in the format: topic: 0 or 1'

    completion = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role":"user", "content":query},
            {"role":"assistant", "content":"'Preparation': 1, 'Location': 0, 'Student discipline': 1, 'Interpersonal relationships': 1, 'Extracurricular/electives': 0, 'Curriculum/curricular materials': 0, 'School choice/enrollment': 0, 'Building quality': 0, 'School safety': 1, 'Teacher quality': 1, 'Facilities': 0, 'Politics': 0, 'Instruction': 0, 'Learning Experience': 1, 'Evaluation': 1, 'Family': 0, 'School staff': 0, 'Other': 0, 'School finances': 0, 'School type': 0, 'Postsecondary/graduation': 0, 'School culture': 1, 'Quality indicator': 0, 'Students': 1, 'Religious': 0, 'Offering': 0."},
            {"role":"user", "content":"explain each answer"}
        ]
    )
    response = completion.choices[0].message.content
    return response
    
response = query_gpt(categories, comment)
response

"'Preparation': 1 - The reviewer mentions that many students are below grade level, which relates to their preparation for the grade they are in.\n\n'Location': 0 - The review does not mention anything about the school's location.\n\n'Student discipline': 1 - The reviewer talks about bullying and fights at the school, indicating a lack of proper student discipline.\n\n'Interpersonal relationships': 1 - The reviewer comments on specific teachers positively, indicating some discussion of interpersonal relationships.\n\n'Extracurricular/electives': 0 - No mention of extracurricular activities or elective subjects was made.\n\n'Curriculum/curricular materials': 0 - There is no explicit discussion about the curriculum or educational materials used at the school.\n\n'School choice/enrollment': 0 - The reviewer does not discuss the process of choosing or being enrolled in the school.\n\n'Building quality': 0 - There is no mention of the physical state or quality of the school building.\n\n'Sc